In [13]:
pip install tqdm


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Data Sourcing

In [20]:
import os
import requests
from tqdm import tqdm
import pandas as pd
import zipfile

In [15]:
# URLs of the zip files
url_202401 = "https://s3.amazonaws.com/tripdata/202401-citibike-tripdata.csv.zip"

In [7]:
# Function to download a file given its URL
def download_file(url, filename):
    # Create the directory if it doesn't exist
    os.makedirs("Data", exist_ok=True)
    
    # Modify filename to include the /data directory
    filename = os.path.join("Data", filename)
    
    if os.path.exists(filename):
        print(f"File '{filename}' already exists.")
        overwrite = input("Do you want to overwrite it? (yes/no): ").lower()
        if overwrite != 'yes':
            print("Skipping download.")
            return

    try:
        # Get the file size for the progress bar
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))

        # Initialize the progress bar
        progress_bar = tqdm(total=total_size, unit='B', unit_scale=True)

        with open(filename, "wb") as f:
            for data in response.iter_content(chunk_size=1024):
                f.write(data)
                progress_bar.update(len(data))

        progress_bar.close()
        print(f"Downloaded {filename}")
    except Exception as e:
        print(f"Error occurred while downloading {filename}: {e}")

In [17]:
download_file(url_202401, "202401-citibike-tripdata.csv.zip")
#alternate code
#download_file(url=url_202401, filename="202401-citibike-tripdata.csv.zip")


### Start Here After Zipfile is Downloaded

In [3]:
# Function to view the contents of a zip file
def view_zip_contents(filename):
    with zipfile.ZipFile(filename, "r") as zip_ref:
        print(f"Contents of {filename}:")
        for file_info in zip_ref.infolist():
            print(file_info.filename)

In [14]:
# View contents of 202401 csv zip file
view_zip_contents("Data/202401-citibike-tripdata.csv.zip")

Contents of Data/202401-citibike-tripdata.csv.zip:
202401-citibike-tripdata_1.csv
__MACOSX/._202401-citibike-tripdata_1.csv
202401-citibike-tripdata_2.csv


In [15]:
def extract_csv_zip_to_dataframe(filename):
    """
    Extract CSV files from a zip file into pandas DataFrames.

    Parameters:
        filename (str): The name of the zip file containing CSV files.
    
    Returns:
        list: A list of pandas DataFrames containing the extracted CSV data.
    """
    dataframes = []
    
    with zipfile.ZipFile(filename, "r") as zip_ref:
        for file_info in zip_ref.infolist():
            if file_info.filename.endswith('.csv'):
                with zip_ref.open(file_info) as file:
                    try:
                        df = pd.read_csv(file)
                    except UnicodeDecodeError:
                        # Try reading with 'latin-1' encoding if UTF-8 fails
                        file.seek(0)  # Reset file pointer
                        df = pd.read_csv(file, encoding='latin-1')
                    dataframes.append(df)
    
    return dataframes

In [16]:
# Extract CSV files from 202401 csv zip file into pandas DataFrames
csv_dataframes = extract_csv_zip_to_dataframe("Data/202401-citibike-tripdata.csv.zip")

/var/folders/7m/f0dfssls4z954t7nqvltms0r0000gq/T/ipykernel_4579/1571236055.py:18: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/var/folders/7m/f0dfssls4z954t7nqvltms0r0000gq/T/ipykernel_4579/1571236055.py:18: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [17]:
# Access the extracted DataFrames
for idx, df in enumerate(csv_dataframes):
    print(f"DataFrame {idx + 1}:")
    print(df.head())  # Print the first few rows of each DataFrame
    print("=" * 50)

DataFrame 1:
            ride_id rideable_type           started_at             ended_at  \
0  58F2CA262B50E256  classic_bike  2024-01-25 20:39:09  2024-01-25 20:44:07   
1  AA7AB6D6E9F8D21B  classic_bike  2024-01-15 18:44:36  2024-01-15 19:19:46   
2  1830A6C4BA1E1A9D  classic_bike  2024-01-03 19:27:58  2024-01-03 19:58:42   
3  3995B084A51A1038  classic_bike  2024-01-22 18:29:46  2024-01-22 18:59:57   
4  23EE6A8979C333B1  classic_bike  2024-01-27 09:55:39  2024-01-27 10:00:48   

    start_station_name start_station_id         end_station_name  \
0   Broadway & E 14 St          5905.12          Ave A & E 11 St   
1  E 16 St & Irving Pl          5938.11  Clermont Ave & Park Ave   
2  E 16 St & Irving Pl          5938.11  Clermont Ave & Park Ave   
3  E 16 St & Irving Pl          5938.11  Clermont Ave & Park Ave   
4  E 16 St & Irving Pl          5938.11          E 14 St & 1 Ave   

  end_station_id  start_lat  start_lng    end_lat    end_lng member_casual  
0        5703.13  40.73454

In [18]:
consolidated_df = pd.concat(csv_dataframes)
# Print the consolidated DataFrame
print(consolidated_df.shape)
consolidated_df.head()

(1954376, 14)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Unnamed: 0
0,58F2CA262B50E256,classic_bike,2024-01-25 20:39:09,2024-01-25 20:44:07,Broadway & E 14 St,5905.12,Ave A & E 11 St,5703.13,40.734546,-73.990741,40.728547,-73.981759,member,NaN
1,AA7AB6D6E9F8D21B,classic_bike,2024-01-15 18:44:36,2024-01-15 19:19:46,E 16 St & Irving Pl,5938.11,Clermont Ave & Park Ave,4692.01,40.735367,-73.987974,40.695734,-73.971297,member,NaN
2,1830A6C4BA1E1A9D,classic_bike,2024-01-03 19:27:58,2024-01-03 19:58:42,E 16 St & Irving Pl,5938.11,Clermont Ave & Park Ave,4692.01,40.735367,-73.987974,40.695734,-73.971297,member,NaN
3,3995B084A51A1038,classic_bike,2024-01-22 18:29:46,2024-01-22 18:59:57,E 16 St & Irving Pl,5938.11,Clermont Ave & Park Ave,4692.01,40.735367,-73.987974,40.695734,-73.971297,member,NaN
4,23EE6A8979C333B1,classic_bike,2024-01-27 09:55:39,2024-01-27 10:00:48,E 16 St & Irving Pl,5938.11,E 14 St & 1 Ave,5779.10,40.735367,-73.987974,40.731393,-73.982867,member,NaN


In [19]:
#drop unneccessary column 
consolidated_df = consolidated_df.drop(columns=['Unnamed: 0'])
consolidated_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,58F2CA262B50E256,classic_bike,2024-01-25 20:39:09,2024-01-25 20:44:07,Broadway & E 14 St,5905.12,Ave A & E 11 St,5703.13,40.734546,-73.990741,40.728547,-73.981759,member
1,AA7AB6D6E9F8D21B,classic_bike,2024-01-15 18:44:36,2024-01-15 19:19:46,E 16 St & Irving Pl,5938.11,Clermont Ave & Park Ave,4692.01,40.735367,-73.987974,40.695734,-73.971297,member
2,1830A6C4BA1E1A9D,classic_bike,2024-01-03 19:27:58,2024-01-03 19:58:42,E 16 St & Irving Pl,5938.11,Clermont Ave & Park Ave,4692.01,40.735367,-73.987974,40.695734,-73.971297,member
3,3995B084A51A1038,classic_bike,2024-01-22 18:29:46,2024-01-22 18:59:57,E 16 St & Irving Pl,5938.11,Clermont Ave & Park Ave,4692.01,40.735367,-73.987974,40.695734,-73.971297,member
4,23EE6A8979C333B1,classic_bike,2024-01-27 09:55:39,2024-01-27 10:00:48,E 16 St & Irving Pl,5938.11,E 14 St & 1 Ave,5779.10,40.735367,-73.987974,40.731393,-73.982867,member


In [ ]:
#zip_ref.extractall("path/to/extract")
#Replace "path/to/extract" with the desired directory path.

## Data Storage Process

In [ ]:
from google.cloud import storage
from google.cloud import bigquery
from google.api_core import exceptions
import os
from io import BytesIO

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Set GOOGLE_APPLICATION_CREDENTIALS environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")


In [ ]:
# Set path to service account key file
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Path"

In [ ]:
print(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))

In [ ]:
# Define the project ID
project_id = 'testingproject-384800'
# Create a client to interact with Google Cloud Storage
client = storage.Client(project=project_id)

In [ ]:
#storing data into a bucket

# Get a specific bucket
bucket_name = 'cis-4400-hw1'
# Check if the bucket exists
bucket = client.bucket(bucket_name)

if not bucket.exists():
    # Create the bucket
    bucket = client.create_bucket(bucket_name)
    print(f'Bucket {bucket.name} created.')
else:
    print(f'Bucket {bucket_name} already exists.')

In [ ]:
# Convert DataFrame to CSV in memory
csv_data = consolidated_df.to_csv(index=False).encode('utf-8')

# Specify the destination blob name in the bucket
destination_blob_name = 'consolidated_df.csv'

# Get a reference to the bucket
bucket = client.bucket(bucket_name)

# Create a blob object
blob = bucket.blob(destination_blob_name)

# Upload the CSV data to the bucket
blob.upload_from_string(csv_data)

print(f"DataFrame uploaded to '{destination_blob_name}' in bucket '{bucket_name}'.")


In [ ]:
# Upload local drive file blob to the bucket


#source_file_name = 'local-file.txt'  # Path to the local file you want to upload
#destination_blob_name = 'uploaded-file.txt'  # Name you want to give to the blob in the bucket
#blob = bucket.blob(destination_blob_name)
#blob.upload_from_filename(source_file_name)
#print("File {} uploaded to {}".format(source_file_name, destination_blob_name))

In [ ]:
# List blobs in the bucket
print("Blobs in bucket '{}':".format(bucket_name))
for blob in bucket.list_blobs():
    print(blob.name)

In [ ]:
# Download a blob from the bucket
#blob_to_download = 'your-blob-to-download.txt'  # Name of the blob you want to download
#destination_file_name = 'downloaded-file.txt'  # Path where you want to save the downloaded file locally
#blob = bucket.blob(blob_to_download)
#blob.download_to_filename(destination_file_name)
#print("Blob {} downloaded to {}".format(blob_to_download, destination_file_name))

### Access Blob Here

In [ ]:
#import 
from google.cloud import storage
from google.cloud import bigquery
from google.api_core import exceptions
import os
from io import BytesIO

In [ ]:
client = storage.Client()

# Specify bucket name and blob name
bucket_name = 'cis-4400-hw1'
blob_name = 'consolidated_df.csv'
blob = bucket.blob(blob_name)

In [ ]:
# List blobs in the bucket
print("Blobs in bucket '{}':".format(bucket_name))
for blob in bucket.list_blobs():
    print(blob.name)

In [ ]:
## Data Warehouse Process 